In [1]:
!nvidia-smi

Sat May 25 12:03:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Импорт библиотек


In [2]:
from google.colab import drive
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders.directory import DirectoryLoader
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch
import numpy as np

In [3]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(DEVICE)

cuda


## Подключение Google Drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# Подготовка данных

## Загрузка книг

In [5]:
# Путь к папке с книгами для базы данных
BOOKS_PATH = '/content/drive/MyDrive/M_QA/books/war_and_peace'

In [6]:
# Извлечь название книги из его пути
def extract_book_name(path):
    base_name = os.path.basename(path)
    return os.path.splitext(base_name)[0]

In [7]:
%%time

# Инициализация загрузчика текстовых документов
loader = DirectoryLoader(BOOKS_PATH, glob="*.txt")
documents = loader.load()

for doc in documents:
    print('Загружена книга:',
          extract_book_name(doc.metadata['source']))

print()

Загружена книга: Война и мир. Том 1
Загружена книга: Война и мир. Том 2
Загружена книга: Война и мир. Том 3
Загружена книга: Война и мир. Том 4

CPU times: user 10.9 s, sys: 734 ms, total: 11.7 s
Wall time: 18.9 s


## Разбивка книг на фрагменты

In [8]:
# Размер чанка (фрагмента текста) в символах
CHUNK_SIZE = 500
# Размер перекрытия чанков (длина пересечений) в символах
CHUNK_OVERLAP = 200

In [9]:
%%time

# Разбиение документов на фрагменты
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    add_start_index=True,
)

chunks = text_splitter.split_documents(documents)
print(f'{len(documents)} книг разбито на {len(chunks)} фрагментов\n')

4 книг разбито на 9900 фрагментов

CPU times: user 813 ms, sys: 4.1 ms, total: 817 ms
Wall time: 817 ms


## Статистика по фрагментам


In [10]:
# Добавление префикса для фрагментов
for c in chunks:
    c.page_content = 'passage: ' + c.page_content

In [11]:
# Тексты всех фрагментов для статистики
contents = [chunk.page_content for chunk in chunks]

In [13]:
# Статистика по длине фрагментов (чанков)
print('min:', np.min(list(map(len, contents))))
print('max:', np.max(list(map(len, contents))))
print('mean:', np.mean(list(map(len, contents))))
print('median:', np.median(list(map(len, contents))))

min: 205
max: 508
mean: 464.4762626262626
median: 504.0


# Добавление фрагментов в Chroma

In [14]:
# Удобная функция для получения модели эмбеддингов
def get_embeddings(model_name):
    model_kwargs = {'device': DEVICE}
    return HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs
    )

In [15]:
%%time

# Создание БД и ее заполнение фрагментами (чанками)
db = Chroma.from_documents(
    documents=chunks,
    collection_name='war_and_peace',
    embedding=get_embeddings('d0rj/e5-base-en-ru'),
    collection_metadata={'hnsw:space': 'cosine'},
    persist_directory='/content/drive/MyDrive/M_QA/chroma/war_and_peace'
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/702 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

CPU times: user 1min 53s, sys: 4.51 s, total: 1min 57s
Wall time: 2min 54s


# Примеры поиска релевантного контекста

In [21]:
# Вопросы, для которых необходимо найти релевантный контекст
questions = [
    'В чем заключается суть философии Пьера Безухова?',
    'Как Наташа реагировала на предложение Андрея Болконского?',
    'Какие чувства испытывала Марья Болконская к князю Андрею?',
    'В каком году русские войска располагались у крепости Бранау?',
    'Где была главная квартира главнокомандующего Кутузова?',
]

In [22]:
%%time

# Поиск трех наиболее релевантных фрагментов к каждому вопросу
results = [db.similarity_search_with_score('query: ' + question, k=3)
           for question in questions]

CPU times: user 111 ms, sys: 1.99 ms, total: 113 ms
Wall time: 117 ms


In [18]:
# Функция для адекватного вывода текстов фрагментов
def wrap_text(text, width=100, indent=''):
    text = text.replace('\n', ' ').replace('\r', '')
    lines = [indent + text[i:i+width] for i in range(0, len(text), width)]
    return '\n'.join(lines)

In [23]:
# Вывод вопросов и найденных контекстов
for i, (question, contexts) in enumerate(zip(questions, results)):
    print(f'Вопрос {i+1}: {question}')
    for j, (context, score) in enumerate(contexts):
        print(f'Контекст {j+1}, близость = {score}:')
        print(wrap_text(context.page_content, width=100, indent='  '))
        source = f'(C) {extract_book_name(context.metadata["source"])}'
        print(f'{source:>102}')
    print()

Вопрос 1: В чем заключается суть философии Пьера Безухова?
Контекст 1, близость = 0.1540234088897705:
  passage: - смерть старого графа Безухого и его наследство. Представьте себе, три княжны получили как
  ую-то малость, князь Василий ничего, а Пьер - наследник всего и, сверх того, признан законным сыном 
  и потому графом Безухим и владельцем самого огромного состояния в России.
                                                                               (C) Война и мир. Том 1
Контекст 2, близость = 0.17130136489868164:
  passage: те из членов, которые казалось были на его стороне, понимали его по своему, с ограничениями
  , изменениями, на которые он не мог согласиться, так как главная потребность Пьера состояла именно в
   том, чтобы передать свою мысль другому точно так, как он сам понимал ее. По окончании заседания вел
  икий мастер с недоброжелательством и иронией сделал Безухому замечание о его горячности и о том, что
   не одна любовь к добродетели, но и увлечение борьбы ру